### 닉네임과 tag를 통해 매치데이터를 임의의 숫자만큼 가져와서 그사람이 플레이한 챔피언들의 데이터프레임 만들기
### 결과컬럼 - [챔피언이름 , 전체게임횟수, 승,패,승률,kills,deaths,assists,kda]

In [28]:
import requests
from tqdm import tqdm
import pandas as pd

In [29]:
name = '괴물쥐'
tag = 'KR3'
api_key = 'RGAPI-bfce5f5d-df88-492d-a316-712d9a8fe9ba'
num = 50
queue = 420

In [30]:
url = f'https://asia.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{name}/{tag}?api_key={api_key}'

In [31]:
puuid=requests.get(url).json()['puuid']

In [32]:
url2 = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue={queue}&start=0&count={num}&api_key={api_key}'

In [35]:
df_create = []
for match_id in tqdm(requests.get(url2).json()):
    url3 = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'
    matches = requests.get(url3).json()
    tmp = []
    try:
        for match in matches['info']['participants']:
            if match['riotIdGameName']== name and match['riotIdTagline'] == tag:
                tmp.append(match['championName'])
                tmp.append(match['win'])
                tmp.append(match['kills'])
                tmp.append(match['deaths'])
                tmp.append(match['assists'])
        df_create.append(tmp)
    except:
        try:
            for match in matches['info']['participants']:
                if match['riotIdName']== name and match['riotIdTagline'] == tag:
                    tmp.append(match['championName'])
                    tmp.append(match['win'])
                    tmp.append(match['kills'])
                    tmp.append(match['deaths'])
                    tmp.append(match['assists'])
            df_create.append(tmp)
        except:
            continue

100%|██████████| 50/50 [00:34<00:00,  1.46it/s]


In [36]:
df = pd.DataFrame(df_create,columns=['championName','win','kills','deaths','assists'])

In [37]:
df

,championName,win,kills,deaths,assists
0,Ezreal,True,5,7,7
1,Ezreal,False,3,6,5
2,Varus,True,13,2,10
3,Kaisa,False,3,6,3
4,Xayah,False,4,4,0
5,Kalista,True,5,0,3
6,Lucian,True,11,0,4
7,TwistedFate,True,9,8,12
8,Nocturne,False,2,2,3
9,Ezreal,True,12,2,6


In [38]:
cnt_df = df[['championName','win']].groupby('championName').count().rename(columns ={'win':'cnt'})

In [39]:
cnt_df

,cnt
championName,
Ashe,1
Caitlyn,4
Ezreal,15
Kaisa,7
Kalista,3
Lucian,2
Milio,1
Nocturne,1
Samira,3


In [40]:
sum_df= df.groupby('championName').sum()

In [44]:
result_df = pd.merge(sum_df,cnt_df,on='championName')

In [47]:
result_df['lose']= result_df.cnt - result_df.win

In [48]:
result_df['win_rate']= round(result_df.win / result_df.cnt,2)*100

In [50]:
result_df['kda']= round((result_df.kills + result_df.assists)/ result_df.deaths,2)

In [53]:
result_df[['cnt','kills','deaths','assists','kda','win','lose','win_rate']].sort_values(by='cnt',ascending=False)

,cnt,kills,deaths,assists,kda,win,lose,win_rate
championName,,,,,,,,
Ezreal,15,115,62,116,3.73,9,6,60.0
Kaisa,7,42,33,33,2.27,2,5,29.0
Xayah,5,46,29,27,2.52,2,3,40.0
Caitlyn,4,21,21,29,2.38,1,3,25.0
Kalista,3,19,18,18,2.06,1,2,33.0
Samira,3,42,21,22,3.05,3,0,100.0
Varus,3,19,7,27,6.57,1,2,33.0
Zeri,3,16,8,18,4.25,2,1,67.0
Lucian,2,26,5,12,7.60,2,0,100.0
